# Lane detection with CARLA

Author: Omar Rodrigo Muñoz Gómez
Date: 2022/11/20

The code creates a vehicle, attaches a camera to it and feeds the video to a UNET for semantic segmentation of the vehicle lanes.

In [1]:
import carla
import math
import random
import time
import cv2
import numpy as np

Define the virtual world

In [3]:
# Create a client to connect to CARLA server
client = carla.Client('localhost', 2000)

# Create a world object to access things within the simulation
town_string = "Town04"
client.load_world(town_string)
world = client.get_world()

In [4]:
# Blue prints give us access to the blueprints needed to create objects
# within CARLA (pedestrians, trees, etc).
bp_lib = world.get_blueprint_library()

# Retrieve the predefined spawn points from the map
spawn_points = world.get_map().get_spawn_points()

Create a vehicle in the virtual world

In [5]:
# Choose a vehicle blueprint and spawn the vehicle at a random location
vehicle_bp = bp_lib.find('vehicle.tesla.model3')
vehicle    = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [6]:
# Position the spectator behind the newly created vehicle
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-5, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

Set the vehicle into autopilot

In [10]:
# Set the autopilot for the vehicles in the world
for v in world.get_actors().filter('vehicle.*.*'):
    v.set_autopilot(True)

Attach a camera into the vehicle

In [7]:
# Create a camera
camera_bp         = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '256')
camera_bp.set_attribute('image_size_y', '128')


camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.4))
camera            = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

Store the last frame into a dictionary

In [8]:
def camera_callback(image, data_dict):
    data_dict['rgb_img'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [9]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

camera_data = {'rgb_img': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

Optionally preview the image from the camera

In [11]:
# Stream the camera feed into an opencv window
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['rgb_img'])
cv2.waitKey(1)

while True:
    cv2.imshow('RGB Camera', camera_data['rgb_img'])
    
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

Load the pretrained UNET model

In [11]:
import tensorflow as tf
from skimage import color

In [12]:
my_model = tf.keras.models.load_model('my_model.h5')

Make predictions over the live video feed

In [13]:
# Resize images to a smaller size
SIZE_X = 128 
SIZE_Y = 128

# Stream the camera feed into an opencv window
cv2.namedWindow('Gray Camera', cv2.WINDOW_AUTOSIZE)
gray = cv2.cvtColor(camera_data['rgb_img'], cv2.COLOR_RGB2GRAY)
cv2.imshow('Gray Camera', gray)
cv2.waitKey(1)

i = 0

while True:
    i = i + 1
    
    
    gray = cv2.cvtColor(camera_data['rgb_img'], cv2.COLOR_RGB2GRAY)
    
    img = cv2.resize(gray, (SIZE_X, SIZE_Y))
    img = tf.cast(img, tf.float32) / 255.0 # Normalize
    img = np.array(img)
    
    y_pred       = my_model.predict(np.expand_dims(img,axis=0))
    y_pred_argmax= np.argmax(y_pred, axis=3)
    res = y_pred_argmax[0]
    
    result_image = color.label2rgb(res, img)    
    
    cv2.imshow('Gray Camera', result_image) # FIXME: Use correct colormap to display
    result_image = cv2.convertScaleAbs(result_image, alpha=(255.0))
    cv2.imwrite('prediction_'+str(i)+'.png', result_image)
    
    
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

1/1 [==============================] - 0s 32ms/step


Optionally, visualize the results of a single frame

In [16]:
# Resize images to a smaller size
SIZE_X = 128 
SIZE_Y = 128

gray = cv2.cvtColor(camera_data['rgb_img'], cv2.COLOR_RGB2GRAY)

img = cv2.resize(gray, (SIZE_X, SIZE_Y))
img = tf.cast(img, tf.float32) / 255.0 # Normalize
img = np.array(img)

y_pred       = my_model.predict(np.expand_dims(img,axis=0))
y_pred_argmax= np.argmax(y_pred, axis=3)
res = y_pred_argmax[0]

result_image = color.label2rgb(res, img)

1/1 [==============================] - 0s 43ms/step


In [17]:
cv2.imshow('sample image',result_image)
 
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image